In [1]:
!pip install -U lightgbm==3.3.2


[notice] A new release of pip is available: 23.1.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install implicit


[notice] A new release of pip is available: 23.1.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip install seaborn


[notice] A new release of pip is available: 23.1.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
from pandas.api.types import CategoricalDtype
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import lightgbm as lgb

import pickle
from tqdm import tqdm
import gc
from pathlib import Path

In [2]:
import warnings
import sys
from IPython.core.interactiveshell import InteractiveShell

warnings.filterwarnings("ignore")
sys.path.append("../") # path to the `src`` folder
InteractiveShell.ast_node_interactivity = "all"
tqdm.pandas()

In [3]:
# !pip install cudf

In [4]:
from src.data import DataHelper
from src.data.metrics import map_at_k, hr_at_k, recall_at_k

from src.retrieval.rules import (
    OrderHistory,
    OrderHistoryDecay,
    ItemPair,
    UserGroupTimeHistory,
    UserGroupSaleTrend,
    TimeHistory,
    TimeHistoryDecay,
    SaleTrend,
    OutOfStock,
    
)

from src.retrieval.rules import *


from src.retrieval.collector import RuleCollector

from src.features import full_sale, week_sale, repurchase_ratio, popularity, period_sale

from src.utils import (
    calc_valid_date,
    merge_week_data,
    reduce_mem_usage,
    calc_embd_similarity,
)

In [5]:
data_dir = Path("../data/")
model_dir = Path("../models/")

In [6]:
TRAIN_WEEK_NUM = 4
WEEK_NUM = TRAIN_WEEK_NUM + 2

VERSION_NAME = "Recall 1"
TEST = True # * Set as `False` when do local experiments to save time

In [7]:
import os
if not os.path.exists(data_dir/"interim"/VERSION_NAME):
    os.makedirs(data_dir/"interim"/VERSION_NAME)
if not os.path.exists(data_dir/"processed"/VERSION_NAME):
    os.makedirs(data_dir/"processed"/VERSION_NAME)

Pepare data: encoding ids and preprocessing

In [8]:
dh = DataHelper(data_dir)

In [8]:
!pip install -U pandas pyarrow


[notice] A new release of pip is available: 23.1.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
!pip install pyarrow fastparquet


[notice] A new release of pip is available: 23.1.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
data = dh.preprocess_data(save=True, name="encoded_full") # * run only once, processed data will be saved

Encode Item Sparse Feats: 100%|██████████| 12/12 [00:00<00:00, 17.26it/s]


In [9]:
data = dh.load_data(name="encoded_full")

In [10]:
uid2idx = pickle.load(open(data_dir/"index_id_map/user_id2index.pkl", "rb"))
submission = pd.read_csv(data_dir/"raw"/'sample_submission.csv')
submission['customer_id'] = submission['customer_id'].map(uid2idx)

## Retrieval


Generate candidates for each week

In [11]:
listBin = [-1, 19, 29, 39, 49, 59, 69, 119]
data['user']['age_bins'] = pd.cut(data['user']['age'], listBin)

In [12]:
df = data['user'].copy()

In [13]:
inter = data["inter"].copy()
users = data["user"][["customer_id", "club_member_status", 'Active']].copy()
numbers = users.groupby(["club_member_status", 'Active']).size()
# Gộp để inter có thông tin Active
inter = inter.merge(users, on="customer_id", how="left")
trans_per_active = inter.groupby(["club_member_status", 'Active']).size()
print(numbers)
print(trans_per_active)


club_member_status  Active
1                   1           5744
                    2            318
2                   1         814039
                    2         458452
3                   1            464
                    2              3
4                   1          87329
                    2           5631
dtype: int64
club_member_status  Active
1                   1            57331
                    2             4834
2                   1         17732495
                    2         13324906
3                   1             8530
                    2                9
4                   1           614112
                    2            46107
dtype: int64


In [14]:
def evaluate_rule(rule_name, rules, trans, customer_list, valid_exploded, week):
    print(f"\n====== TEST RULE: {rule_name} (week {week}) ======")

    candidates = RuleCollector().collect(
        week_num = week,
        trans_df = trans,
        customer_list=customer_list,
        rules=rules,
        min_pos_rate=0.000,
        compress=False,
    )

    # pivot
    candidates = (
        pd.pivot_table(
            candidates,
            values="score",
            index=["customer_id", "article_id"],
            columns=["method"],
            aggfunc=np.sum,
        )
        .reset_index()
    )

    # ---- CALCULATE PRECISION & RECALL ----
    matched = candidates.merge(valid_exploded, on=['customer_id','article_id'], how='inner')
    num_matched = len(matched)
    total_valid = len(valid_exploded)

    recall = num_matched / total_valid
    precision = num_matched / len(candidates)

    result = {
        "rule": rule_name,
        "num_candidates": len(candidates),
        "num_matched": num_matched,
        "precision": precision,
        "recall": recall
    }

    print(result)
    return result, candidates


In [15]:
DAYS = [15, 20, 25, 30]
CAT_COLS = [["product_type_no"], ["product_type_no", "colour_group_code"]]
N = [8, 10, 12]

In [16]:
all_results = []

for week in range(1, WEEK_NUM):

    trans = data["inter"]
    items = data["item"]

    start_date, end_date = calc_valid_date(week)
    train, valid = dh.split_data(trans, start_date, end_date)

    # merge user features
    customer_cols = ['customer_id', 'age_bins', 'club_member_status', 'Active']
    train = train.merge(
        data['user'][customer_cols],
        on='customer_id',
        how='left'
    )

    # giảm RAM
    train["customer_id"] = train["customer_id"].astype("category")

    valid_exploded = valid.explode('article_id')
    customer_list = valid["customer_id"].values

    print(f"\n========= WEEK {week} | valid size = {len(valid_exploded)} =========")

    # 🔁 sweep tham số
    for cat_cols in CAT_COLS:
        for days in DAYS:
            for n in N:

                rule_name = f"IGTH_{days}_{n}_{'_'.join(cat_cols)}"

                rule = ItemGroupTimeHistory(
                    customer_list=customer_list,
                    trans=train,
                    item_df=items,
                    cat_cols=cat_cols,
                    days=days,
                    n=n,
                    name="1"
                )

                result, candidates = evaluate_rule(
                    rule_name=rule_name,
                    rules=[rule],
                    trans=trans,
                    customer_list=customer_list,
                    valid_exploded=valid_exploded,
                    week=week
                )

                # lưu metric
                all_results.append(result)

                # lưu candidates
                candidates.to_parquet(
                    data_dir / "interim" / VERSION_NAME /
                    f"week{week}_{rule_name}_candidates.pqt"
                )

                del candidates, rule

    valid.to_parquet(
        data_dir / "processed" / VERSION_NAME / f"week{week}_label.pqt"
    )
df_result = pd.DataFrame(all_results)

df_result.to_csv(
    data_dir / "interim" / VERSION_NAME /
    "item_group_time_history_results.csv",
    index=False
)

df_result.sort_values("precision", ascending=False)


========= WEEK 1 | valid size = 240311 =========

====== TEST RULE: IGTH_15_8_product_type_no (week 1) ======


Retrieve items by rules: 100%|██████████| 1/1 [00:19<00:00, 19.52s/it]

Positive rate: 0.00233


MemoryError: Unable to allocate 7.63 GiB for an array with shape (1024315295,) and data type int64

In [ ]:
# all_results = []

# for week in range(1, WEEK_NUM):

#     trans = data["inter"]
#     items = data["item"]
#     start_date, end_date = calc_valid_date(week)
#     train, valid = dh.split_data(trans, start_date, end_date)
#     customer_cols = ['customer_id', 'age_bins', 'club_member_status', 'Active']

#     item_cols = ['product_type_no']

#     train = train.merge(data['user'][customer_cols], on='customer_id', how='left')

    
#     last_week_start = (pd.to_datetime(start_date) - pd.Timedelta(days=7)).strftime("%Y-%m-%d")
#     last_week = train.loc[train.t_dat >= last_week_start]

#     last_3day_start = (pd.to_datetime(start_date) - pd.Timedelta(days=3)).strftime("%Y-%m-%d")
#     last_3days = train.loc[train.t_dat >= last_3day_start]

#     valid_exploded = valid.explode('article_id')
#     customer_list = valid["customer_id"].values
    
#     RULE_SET = {
#         "IGST_7_50": [
#             ItemGroupSaleTrend(customer_list, train, items, ["product_type_no"], days=7, n=20, t=0.5, name='1'),
#         ]
#     }


#     print(f"\n========= WEEK {week} | valid size = {len(valid_exploded)} =========")

#     # ⚡ Chạy lần lượt từng rule
#     for rule_name, rule_obj in RULE_SET.items():
#         result, candidates = evaluate_rule(
#             rule_name=rule_name,
#             rules=rule_obj,
#             trans=trans,
#             customer_list=customer_list,
#             valid_exploded=valid_exploded,
#             week=week
#         )

#         # lưu result
#         all_results.append(result)

#         # xuất candidate từng rule
#         candidates.to_parquet(
#             data_dir/"interim"/VERSION_NAME/f"week{week}_{rule_name}_candidates.pqt"
#         )

#     valid.to_parquet(data_dir/"processed"/VERSION_NAME/f"week{week}_label.pqt")

# df_result = pd.DataFrame(all_results)
# df_result.to_csv(data_dir/"interim"/VERSION_NAME/"all_rule_results.csv", index=False)
# df_result.sort_values("precision", ascending=False)



========= WEEK 1 | valid size = 240311 =========

====== TEST RULE: IGST_7_50 (week 1) ======


Retrieve items by rules: 100%|██████████| 1/1 [00:28<00:00, 28.59s/it]

Positive rate: 0.00004


{'rule': 'IGST_7_50', 'num_candidates': 5955406, 'num_matched': 313, 'precision': 5.255728996478158e-05, 'recall': 0.0013024788711294946}

========= WEEK 2 | valid size = 255241 =========

====== TEST RULE: IGST_7_50 (week 2) ======


Retrieve items by rules: 100%|██████████| 1/1 [00:27<00:00, 27.13s/it]

Positive rate: 0.00004


{'rule': 'IGST_7_50', 'num_candidates': 6261137, 'num_matched': 284, 'precision': 4.535917358141181e-05, 'recall': 0.0011126739042708657}

========= WEEK 3 | valid size = 264657 =========

====== TEST RULE: IGST_7_50 (week 3) ======


Retrieve items by rules: 100%|██████████| 1/1 [00:33<00:00, 33.83s/it]


Positive rate: 0.00003


KeyboardInterrupt: 

In [ ]:
# * use the threshold in week 1 to generate candidates for test data, see the log in the upper cell 
if TEST:
    week = 0
    trans = data["inter"]
    
    start_date, end_date = calc_valid_date(week)
    print(f"Week {week}: [{start_date}, {end_date})")
    
    train, valid = dh.split_data(trans, start_date, end_date)
    train = train.merge(data['user'][['customer_id','age_bins']], on='customer_id', how='left')

    last_week_start = pd.to_datetime(start_date) - pd.Timedelta(days=7)
    last_week_start = last_week_start.strftime("%Y-%m-%d")
    last_week = train.loc[train.t_dat >= last_week_start]
    
    last_3day_start = pd.to_datetime(start_date) - pd.Timedelta(days=3)
    last_3day_start = last_3day_start.strftime("%Y-%m-%d")
    last_3days = train.loc[train.t_dat >= last_3day_start]

    customer_list = submission['customer_id'].values

    # * ========================== Retrieval Strategies ==========================

    candidates = RuleCollector().collect(
        week_num = week,
        trans_df = trans,
        customer_list=customer_list,
        rules=[
            OrderHistory(train, days=3, name='1'),
            OrderHistory(train, days=7, name='2'),
            OrderHistoryDecay(train, days=3, n=50, name='1'),
            OrderHistoryDecay(train, days=7, n=50, name='2'),
            ItemPair(OrderHistory(train, days=3).retrieve(), name='1'),
            ItemPair(OrderHistory(train, days=7).retrieve(), name='2'),
            ItemPair(OrderHistoryDecay(train, 3, n=50).retrieve(), name='3'),
            ItemPair(OrderHistoryDecay(train, 7, n=50).retrieve(), name='4'),
            UserGroupTimeHistory(data, customer_list, last_week, ['age_bins'], n=15, name='1'),
            UserGroupTimeHistory(data, customer_list, last_3days, ['age_bins'], n=20.5, name='2'),
            UserGroupSaleTrend(data, customer_list, train, ['age_bins'], days=7, n=2),
            TimeHistory(customer_list, last_week, n=9, name='1'),
            TimeHistory(customer_list, last_3days, n=16, name='2'),
            TimeHistoryDecay(customer_list, train, days=3, n=12, name='1'),
            TimeHistoryDecay(customer_list, train, days=7, n=8, name='2'),
            SaleTrend(customer_list, train, days=7, n=2),
        ],
        filters=[OutOfStock(trans)],
        min_pos_rate=0.006,
        compress=False,
    )
    
    candidates, _ = reduce_mem_usage(candidates)
    candidates = (
        pd.pivot_table(
            candidates,
            values="score",
            index=["customer_id", "article_id"],
            columns=["method"],
            aggfunc=np.sum,
        )
        .reset_index()
    )

    candidates.to_parquet(data_dir/"interim"/VERSION_NAME/f"week{week}_candidate.pqt")
    valid.to_parquet(data_dir/"processed"/VERSION_NAME/f"week{week}_label.pqt")

Week 0: [2020-09-23, 2020-09-30)


KeyboardInterrupt: 

In [ ]:
del train, valid, last_week, customer_list, candidates
gc.collect()

## Feature engineering


In [ ]:
user = data["user"]
item = data["item"]
inter = data["inter"]

In [ ]:
# calculate week number
inter['week'] = (pd.to_datetime('2020-09-29') - pd.to_datetime(inter['t_dat'])).dt.days // 7

In [ ]:
# merge full candidates to transaction data (avoid feature missing in training data)
full_candidates = []
for i in tqdm(range(WEEK_NUM)):
    candidate = pd.read_parquet(data_dir/"interim"/VERSION_NAME/f"week{i}_candidate.pqt")
    full_candidates += candidate['article_id'].values.tolist()
full_candidates = list(set(full_candidates))
del candidate
gc.collect()

num_candidates = len(full_candidates)
full_candidates = np.array(full_candidates)
full_candidates = np.tile(full_candidates, WEEK_NUM + 1)
weeks = np.repeat(np.arange(1,WEEK_NUM+2), num_candidates)
full_candidates = pd.DataFrame({'article_id':full_candidates, 'week':weeks})

inter['valid'] = 1
in_train = inter[inter['week']<=WEEK_NUM + 1]
out_train = inter[inter['week']>WEEK_NUM + 1]

in_train = in_train.merge(full_candidates, on=['article_id','week'], how='right')
in_train['valid'] = in_train['valid'].fillna(0)
inter = pd.concat([in_train, out_train], ignore_index=True)
inter = inter.sort_values(["valid"], ascending=False).reset_index(drop=True)

100%|██████████| 6/6 [00:18<00:00,  3.07s/it]


17

In [ ]:
# merge `product_code`
inter = inter.merge(item[["article_id", "product_code"]], on="article_id", how="left")

In [ ]:
inter.shape

(31837313, 8)

In [ ]:
_, inter["i_1w_sale_rank"], inter["i_1w_sale_norm"] = period_sale(
    inter, ["article_id"], days=14, rank=True, norm=True, week_num=WEEK_NUM
)
_, inter["p_1w_sale_rank"], inter["p_1w_sale_norm"] = period_sale(
    inter, ["product_code"], days=14, rank=True, norm=True, week_num=WEEK_NUM
)
inter["i_2w_sale"], inter["i_2w_sale_rank"], inter["i_2w_sale_norm"] = period_sale(
    inter, ["article_id"], days=14, rank=True, norm=True, week_num=WEEK_NUM
)
inter["p_2w_sale"], inter["p_2w_sale_rank"], inter["p_2w_sale_norm"] = period_sale(
    inter, ["product_code"], days=14, rank=True, norm=True, week_num=WEEK_NUM
)

In [ ]:
inter["i_3w_sale"], inter["i_3w_sale_rank"], inter["i_3w_sale_norm"] = period_sale(
    inter, ["article_id"], days=21, rank=True, norm=True, week_num=WEEK_NUM
)
inter["p_3w_sale"], inter["p_3w_sale_rank"], inter["p_3w_sale_norm"] = period_sale(
    inter, ["product_code"], days=21, rank=True, norm=True, week_num=WEEK_NUM
)
inter["i_4w_sale"], inter["i_4w_sale_rank"], inter["i_4w_sale_norm"] = period_sale(
    inter, ["article_id"], days=28, rank=True, norm=True, week_num=WEEK_NUM
)
inter["p_4w_sale"], inter["p_4w_sale_rank"], inter["p_4w_sale_norm"] = period_sale(
    inter, ["product_code"], days=28, rank=True, norm=True, week_num=WEEK_NUM
)

In [ ]:
inter.shape

(31837313, 30)

In [ ]:
inter['i_repurchase_ratio'] = repurchase_ratio(inter, ['article_id'], week_num=WEEK_NUM)
inter['p_repurchase_ratio'] = repurchase_ratio(inter, ['product_code'], week_num=WEEK_NUM)

100%|██████████| 6/6 [02:00<00:00, 20.07s/it]


In [ ]:
inter.shape

(31837313, 32)

In [ ]:
inter, _ = reduce_mem_usage(inter)

In [ ]:
inter["i_sale"] = week_sale(inter, ["article_id"], week_num=WEEK_NUM)
inter["p_sale"] = week_sale(inter, ["product_code"], week_num=WEEK_NUM)
inter["i_sale_uni"] = week_sale(inter, ["article_id"], True, week_num=WEEK_NUM)
inter["p_sale_uni"] = week_sale(inter, ["product_code"], True, week_num=WEEK_NUM)
inter["lw_i_sale"] = week_sale(inter, ["article_id"], step=1, week_num=WEEK_NUM) # * last week sale
inter["lw_p_sale"] = week_sale(inter, ["product_code"], step=1, week_num=WEEK_NUM)
inter["lw_i_sale_uni"] = week_sale(inter, ["article_id"], True, step=1, week_num=WEEK_NUM)
inter["lw_p_sale_uni"] = week_sale(inter, ["product_code"], True, step=1, week_num=WEEK_NUM)

inter["i_sale_ratio"] = inter["i_sale"] / (inter["p_sale"] + 1e-6)
inter["i_sale_uni_ratio"] = inter["i_sale_uni"] / (inter["p_sale_uni"] + 1e-6)
inter["lw_i_sale_ratio"] = inter["lw_i_sale"] / (inter["lw_p_sale"] + 1e-6)
inter["lw_i_sale_uni_ratio"] = inter["lw_i_sale_uni"] / (inter["lw_p_sale_uni"] + 1e-6)

inter["i_uni_ratio"] = inter["i_sale"] / (inter["i_sale_uni"] + 1e-6)
inter["p_uni_ratio"] = inter["p_sale"] / (inter["p_sale_uni"] + 1e-6)
inter["lw_i_uni_ratio"] = inter["lw_i_sale"] / (inter["lw_i_sale_uni"] + 1e-6)
inter["lw_p_uni_ratio"] = inter["lw_p_sale"] / (inter["lw_p_sale_uni"] + 1e-6)

inter["i_sale_trend"] = (inter["i_sale"] - inter["lw_i_sale"]) / (inter["lw_i_sale"] + 1e-6)
inter["p_sale_trend"] = (inter["p_sale"] - inter["lw_p_sale"]) / (inter["lw_p_sale"] + 1e-6)

item_feats = [
    "product_type_no",
    # "product_group_name",
    # "graphical_appearance_no",
    # "colour_group_code",
    # "perceived_colour_value_id",
    # "perceived_colour_master_id",
]
inter = inter.merge(item[["article_id", *item_feats]], on="article_id", how="left")

for f in tqdm(item_feats):
    inter[f"{f}_sale"] = week_sale(inter, [f], f"{f}_sale", week_num=WEEK_NUM)
    inter[f"lw_{f}_sale"] = week_sale(inter, [f], f"{f}_sale", step=1, week_num=WEEK_NUM)
    inter[f"{f}_sale_trend"] = (inter[f"{f}_sale"] - inter[f"lw_{f}_sale"]) / (inter[f"lw_{f}_sale"] + 1e-6)

100%|██████████| 1/1 [00:22<00:00, 22.43s/it]


In [ ]:
inter.shape

(31837313, 54)

In [ ]:
# * Date related
curr_date_dict = {x:calc_valid_date(x-1)[0] for x in range(100)}
current_dat = inter['week'].map(curr_date_dict)
mask = inter['valid']==0
inter.loc[mask, 't_dat'] = inter.loc[mask, 'week'].map(curr_date_dict)
first_date = inter.groupby('article_id')['t_dat'].min().reset_index(name='first_dat')
inter = pd.merge(inter, first_date, on='article_id', how='left')
# df = pd.merge(df, last_date, on='article_id', how='left')
inter['first_dat'] = (pd.to_datetime(current_dat)-pd.to_datetime(inter['first_dat'])).dt.days

In [ ]:
inter.shape

(31837313, 55)

In [ ]:
inter['i_full_sale'] = full_sale(inter, ['article_id'], week_num=WEEK_NUM)
inter['p_full_sale'] = full_sale(inter, ['product_code'], week_num=WEEK_NUM)

inter['i_daily_sale'] = inter['i_full_sale'] / inter['first_dat']
inter['p_daily_sale'] = inter['p_full_sale'] / inter['first_dat']
inter['i_daily_sale_ratio'] = inter['i_daily_sale'] / inter['p_daily_sale']
inter['i_w_full_sale_ratio'] = inter['i_sale'] / inter['i_full_sale']

inter['i_2w_full_sale_ratio'] = inter['i_2w_sale'] / inter['i_full_sale']
inter['p_w_full_sale_ratio'] = inter['p_sale'] / inter['p_full_sale']
inter['p_2w_full_sale_ratio'] = inter['p_2w_sale'] / inter['p_full_sale']

inter['i_week_above_daily_sale'] = inter['i_sale'] / 7 - inter['i_daily_sale']
inter['p_week_above_full_sale'] = inter['p_sale'] / 7 - inter['i_full_sale']
inter['i_2w_week_above_daily_sale'] = inter['i_2w_sale'] / 14 - inter['i_daily_sale']
inter['p_2w_week_above_daily_sale'] = inter['p_2w_sale'] / 14 - inter['p_daily_sale']

In [ ]:
gc.collect()

94

In [ ]:
for f in tqdm(item_feats):
    inter[f'{f}_full_sale'] = full_sale(inter, [f], week_num=WEEK_NUM)
    f_first_date = inter.groupby(f)['t_dat'].min().reset_index(name=f'{f}_first_dat')
    inter = inter.merge(f_first_date, on=f, how='left')
    inter[f'{f}_daily_sale'] = inter[f'{f}_full_sale'] / (pd.to_datetime(current_dat) - pd.to_datetime(inter[f'{f}_first_dat'])).dt.days
    inter[f'i_{f}_daily_sale_ratio'] = inter['i_daily_sale'] / inter[f'{f}_daily_sale']
    inter[f'p_{f}_daily_sale_ratio'] = inter['p_daily_sale'] / inter[f'{f}_daily_sale']
    del inter[f'{f}_full_sale'], inter[f'{f}_first_dat']
    gc.collect()

  0%|          | 0/1 [00:00<?, ?it/s]

17

100%|██████████| 1/1 [01:21<00:00, 81.40s/it]


In [ ]:
for f in item_feats + ['i_full_sale','p_full_sale']:
    del inter[f]

In [ ]:
inter['i_pop'] = popularity(inter, 'article_id', week_num=WEEK_NUM)
inter['p_pop'] = popularity(inter, 'product_code', week_num=WEEK_NUM)

In [ ]:
inter = inter.loc[inter['week'] <= WEEK_NUM + 2]

In [ ]:
inter.to_parquet(data_dir / "processed/processed_inter.pqt")

## Merge Features


In [ ]:
inter = pd.read_parquet(data_dir / "processed/processed_inter.pqt")
inter = inter[inter['week'] <= WEEK_NUM + 2]

In [ ]:
#* embeddings from DSSM model
dssm_user_embd = np.load(data_dir / "external/dssm_user_embd.npy", allow_pickle=True)
dssm_item_embd = np.load(data_dir / "external/dssm_item_embd.npy", allow_pickle=True)
# * embeddings from YouTubeDNN model
yt_user_embd = np.load(data_dir / "external/yt_user_embd.npy", allow_pickle=True)
yt_item_embd = np.load(data_dir / "external/yt_item_embd.npy", allow_pickle=True)
# * embeddings from Word2Vector model
w2v_user_embd = np.load(data_dir/'external'/'w2v_user_embd.npy', allow_pickle=True)
w2v_item_embd = np.load(data_dir/'external'/'w2v_item_embd.npy', allow_pickle=True)

In [ ]:
for col in inter.columns:
    inter[col] = np.nan_to_num(inter[col])

In [ ]:
for i in tqdm(range(WEEK_NUM)):
    if i == 0 and not TEST:
        continue
    candidate = pd.read_parquet(data_dir/"interim"/VERSION_NAME/f"week{i}_candidate.pqt")
    if i == 0:
        chunk_size = int(candidate.shape[0] * 0.5)
        for chunk,batch in enumerate(range(0, candidate.shape[0], chunk_size)):
            sub_candidate = candidate.iloc[batch:batch+chunk_size-1]
            # * merge features
            sub_candidate = merge_week_data(data, inter, i, sub_candidate)
            sub_candidate['article_id'] = sub_candidate['article_id'].astype(int)
            sub_candidate['customer_id'] = sub_candidate['customer_id'].astype(int)
            # * merge DSSM user and item embeddings
            sub_candidate["dssm_similarity"] = calc_embd_similarity(sub_candidate, dssm_user_embd, dssm_item_embd)
            # * merge YouTubeDNN user and item embeddings
            sub_candidate["yt_similarity"] = calc_embd_similarity(sub_candidate, yt_user_embd, yt_item_embd)
            # * merge Word2Vector user and item embeddings
            sub_candidate["wv_similarity"] = calc_embd_similarity(sub_candidate, w2v_user_embd, w2v_item_embd, sub=False)
            print(f"Chunk {chunk} done...")
            sub_candidate.to_parquet(data_dir/"processed"/VERSION_NAME/f"week{i}_candidate_{chunk}.pqt")
    else:
        # * merge features
        candidate = merge_week_data(data, inter, i, candidate)
        print(candidate['week'].unique())
        # * merge DSSM user and item embeddings
        candidate["dssm_similarity"] = calc_embd_similarity(candidate, dssm_user_embd, dssm_item_embd)
        # * merge YouTubeDNN user and item embeddings
        candidate["yt_similarity"] = calc_embd_similarity(candidate, yt_user_embd, yt_item_embd)
        candidate["wv_similarity"] = calc_embd_similarity(candidate, w2v_user_embd, w2v_item_embd, sub=False)
    candidate.to_parquet(data_dir/"processed"/VERSION_NAME/f"week{i}_candidate.pqt")

100%|██████████| 2766/2766 [00:10<00:00, 258.02it/s]

100%|██████████| 2766/2766 [00:10<00:00, 257.38it/s]

100%|██████████| 2766/2766 [00:31<00:00, 87.67it/s] 


Chunk 0 done...



100%|██████████| 2766/2766 [00:11<00:00, 240.03it/s]

100%|██████████| 2766/2766 [00:11<00:00, 242.01it/s]

100%|██████████| 2766/2766 [00:22<00:00, 123.16it/s]


Chunk 1 done...


 17%|█▋        | 1/6 [11:46<58:51, 706.31s/it]

...


100%|██████████| 280/280 [00:01<00:00, 213.52it/s]

100%|██████████| 280/280 [00:01<00:00, 236.83it/s]

 67%|██████▋   | 4/6 [13:46<04:19, 129.64s/it]

80253
[4]



100%|██████████| 298/298 [00:01<00:00, 207.64it/s]

100%|██████████| 298/298 [00:01<00:00, 222.65it/s]

 83%|████████▎ | 5/6 [14:29<01:38, 98.39s/it] 

72035
[5]



100%|██████████| 178/178 [00:00<00:00, 203.60it/s]

100%|██████████| 178/178 [00:00<00:00, 220.95it/s]

100%|██████████| 6/6 [15:14<00:00, 152.45s/it]


In [ ]:
del dssm_user_embd, dssm_item_embd, yt_user_embd, yt_item_embd
gc.collect()

858

## Ranking


In [ ]:
candidates = {}
labels = {}
for i in tqdm(range(1, WEEK_NUM)):
    candidates[i] = pd.read_parquet(data_dir/"processed"/VERSION_NAME/f"week{i}_candidate.pqt")
    labels[i] = pd.read_parquet(data_dir/"processed"/VERSION_NAME/f"week{i}_label.pqt")    

100%|██████████| 5/5 [00:06<00:00,  1.36s/it]


In [ ]:
feats = [
    x
    for x in candidates[1].columns
    if x
    not in [
        "label",
        "sales_channel_id",
        "t_dat",
        "week",
    ]
]
cat_features = [
    "customer_id",
    "article_id",
    "product_code",
    "FN",
    "Active",
    "club_member_status",
    "fashion_news_frequency",
    "age",
    "product_type_no",
    "product_group_name",
    "graphical_appearance_no",
    "colour_group_code",
    "perceived_colour_value_id",
    "perceived_colour_master_id",

    "user_gender",
    "article_gender",
    "season_type"
]

In [ ]:
# * Convert categorical featues as `CategoricalDtype`
cate_dict = {}        
for feat in tqdm(cat_features):
    if feat in data['user'].columns:
        value_set = set(data['user'][feat].unique())
    elif feat in data['item'].columns:
        value_set = set(data['item'][feat].unique())
    else:
        value_set = set(data['inter'][feat].unique())
    cate_dict[feat] = CategoricalDtype(categories=value_set)

100%|██████████| 17/17 [00:00<00:00, 18.05it/s]


In [ ]:
full_data = pd.concat([candidates[i] for i in range(1, WEEK_NUM)], ignore_index=True)

### Extra Features

In [ ]:
inter = data['inter']
inter = inter[inter['t_dat']<'2020-08-19'] # * start date of the last valid week
inter['week'] = (pd.to_datetime('2020-09-29') - pd.to_datetime(inter['t_dat'])).dt.days // 7
inter = inter.merge(data['item'][["article_id", "product_code"]], on="article_id", how="left")

In [ ]:
tmp = inter.groupby('article_id').week.mean()
full_data['article_time_mean'] = full_data['article_id'].map(tmp)

tmp = inter.groupby('customer_id').week.nth(-1)
full_data['customer_id_last_time'] = full_data['customer_id'].map(tmp)

tmp = inter.groupby('customer_id').week.nth(0)
full_data['customer_id_first_time'] = full_data['customer_id'].map(tmp)

tmp = inter.groupby('customer_id').week.mean()
full_data['customer_id_time_mean'] = full_data['customer_id'].map(tmp)

full_data['customer_id_gap'] = full_data['customer_id_first_time'] - full_data['customer_id_last_time']

In [ ]:
feats += [
    'article_time_mean', 
    'customer_id_last_time', 
    'customer_id_first_time', 
    'customer_id_time_mean',
    'customer_id_gap'
]

In [ ]:
del tmp
gc.collect()

450

### Train


In [ ]:
for feat in tqdm(cat_features):
    full_data[feat] = full_data[feat].astype(cate_dict[feat])

100%|██████████| 17/17 [00:10<00:00,  1.66it/s]


In [ ]:
train = full_data.loc[full_data['week']>1]
valid = full_data.loc[full_data['week']==1]

del full_data
gc.collect()

88

In [ ]:
params = {
    "objective": "binary",#"lambdarank",
    "boosting_type": "gbdt",
    "metric": "auc",#"map",
    "max_depth": 8,
    "num_leaves": 128,
    "learning_rate": 0.03,

    "verbose": -1,
    "eval_at": 12,
}

In [ ]:
def train_rank_model(train, valid, train_group, valid_group):

    train_set = lgb.Dataset(
        data=train[feats],
        label=train["label"],
        group=train_group,
        feature_name=feats,
        categorical_feature=cat_features,
        params=params,
    )

    valid_set = lgb.Dataset(
        data=valid[feats],
        label=valid["label"],
        group=valid_group,
        feature_name=feats,
        categorical_feature=cat_features,
        params=params,
    )

    ranker = lgb.train(
        params,
        train_set,
        num_boost_round=300,
        valid_sets=[valid_set],
        early_stopping_rounds=30,
        verbose_eval=10,
    )
    ranker.save_model(
        model_dir / f"lgb_small_ranker.model",
        num_iteration=ranker.best_iteration,
    )
    return ranker

In [ ]:
def train_binary_model(train, valid):

    train_set = lgb.Dataset(
        data=train[feats],
        label=train["label"],
        feature_name=feats,
        categorical_feature=cat_features,
        params=params,
    )

    valid_set = lgb.Dataset(
        data=valid[feats],
        label=valid["label"],
        feature_name=feats,
        categorical_feature=cat_features,
        params=params,
    )

    ranker = lgb.train(
        params,
        train_set,
        num_boost_round=300,
        valid_sets=[valid_set],
        early_stopping_rounds=30,
        verbose_eval=10,
    )
    ranker.save_model(
        model_dir / f"lgb_small_binary.model",
        num_iteration=ranker.best_iteration,
    )
    return ranker

In [ ]:
del candidates
gc.collect()

568

In [ ]:
print("Train positive rate:", train.label.mean())

Train positive rate: 0.0067018773363074985


In [ ]:
train = train.sort_values(by=["week", "customer_id"], ascending=True).reset_index(drop=True)
valid = valid.sort_values(by=["customer_id"], ascending=True).reset_index(drop=True)

In [ ]:
train_group = train[["customer_id", "article_id", "week"]]
train_group = train_group.astype("int32")  # * convert to int to avoid `0` in groupby count result
train_group = (train_group.groupby(["week", "customer_id"]).size().values)

In [ ]:
valid_group = valid[["customer_id", "article_id"]]
valid_group = valid_group.astype("int32")  # * convert to int to avoid `0` in groupby count result
valid_group = valid_group.groupby(["customer_id"]).size().values

In [ ]:
train = train[feats+['label']]
valid = valid[feats+['label']]

In [ ]:
gc.collect()

247

In [ ]:
# ranker = train_rank_model(train, valid, train_group, valid_group)
ranker = train_binary_model(train, valid)

Training until validation scores don't improve for 30 rounds
[10]	valid_0's auc: 0.690798
[20]	valid_0's auc: 0.699496
[30]	valid_0's auc: 0.703928
[40]	valid_0's auc: 0.706298
[50]	valid_0's auc: 0.708892
[60]	valid_0's auc: 0.709672
[70]	valid_0's auc: 0.710949
[80]	valid_0's auc: 0.713591
[90]	valid_0's auc: 0.715312
[100]	valid_0's auc: 0.716153
[110]	valid_0's auc: 0.717036
[120]	valid_0's auc: 0.717173
[130]	valid_0's auc: 0.717293
[140]	valid_0's auc: 0.717737
[150]	valid_0's auc: 0.717838
[160]	valid_0's auc: 0.717651
[170]	valid_0's auc: 0.717697
Early stopping, best iteration is:
[148]	valid_0's auc: 0.717971


### Inference

In [ ]:
# ranker = lgb.Booster(model_file=model_dir / "lgb_small_binary.model")
ranker = lgb.Booster(model_file=model_dir / "lgb_small_ranker.model")

In [ ]:
feat_importance = pd.DataFrame(
    {"feature": feats, "importance": ranker.feature_importance()}
).sort_values(by="importance", ascending=False)
plt.figure(figsize=(8, 22))
sns.barplot(y="feature", x="importance", data=feat_importance)

### Validate

In [ ]:
val_candidates = valid.reset_index(drop=True)

In [ ]:
def predict(ranker, candidates, batch_size = 5_000_000):
    probs = np.zeros(candidates.shape[0])
    for batch in range(0, candidates.shape[0], batch_size):
        outputs = ranker.predict(candidates.loc[batch : batch + batch_size - 1, feats])
        probs[batch : batch + batch_size] = outputs
    candidates["prob"] = probs
    pred_lgb = candidates[['customer_id','article_id','prob']]
    pred_lgb = pred_lgb.sort_values(by=["customer_id","prob"], ascending=False).reset_index(drop=True)
    pred_lgb.rename(columns={'article_id':'prediction'}, inplace=True)
    pred_lgb = pred_lgb.drop_duplicates(['customer_id', 'prediction'], keep='first')
    pred_lgb['customer_id'] = pred_lgb['customer_id'].astype(int)
    pred_lgb = pred_lgb.groupby("customer_id")["prediction"].progress_apply(list).reset_index()
    return pred_lgb

In [ ]:
pred = predict(ranker, val_candidates)

100%|██████████| 68984/68984 [00:06<00:00, 10932.32it/s]


In [ ]:
label = labels[1]
label = pd.merge(label, pred, on="customer_id", how="left")

In [ ]:
map_at_k(label["article_id"], label["prediction"], k=12)

# 0.029813727108367518 ranker
# 0.029791925075924913 binary

0.029791925075924913

In [ ]:
batch_size = 5_000_000
probs = np.zeros(val_candidates.shape[0])
for batch in range(0, val_candidates.shape[0], batch_size):
    outputs = ranker.predict(val_candidates.loc[batch : batch + batch_size - 1, feats])
    probs[batch : batch + batch_size] = outputs
val_candidates["prob"] = probs
pred_lgb = val_candidates[['customer_id','article_id','prob']]
pred_lgb = pred_lgb.sort_values(by=["customer_id","prob"], ascending=False).reset_index(drop=True)
pred_lgb.rename(columns={'article_id':'prediction'}, inplace=True)
pred_lgb = pred_lgb.drop_duplicates(['customer_id', 'prediction'], keep='first')
pred_lgb['customer_id'] = pred_lgb['customer_id'].astype(int)

In [ ]:
pred_lgb.to_parquet(data_dir/"processed"/"small_binary_valid.pqt")

### Test

In [ ]:
del candidates
gc.collect()

414

In [ ]:
test_pred = []
for chunk in range(2):
    print(f"Chunk {chunk}")
    test_candidates = pd.read_parquet(data_dir/"processed"/VERSION_NAME/f"week0_candidate_{chunk}.pqt")
    for feat in cat_features:
        test_candidates[feat] = test_candidates[feat].astype(cate_dict[feat])

    # * Extra Features ===================================

    tmp = inter.groupby('article_id').week.mean()
    test_candidates['article_time_mean'] = test_candidates['article_id'].map(tmp)

    tmp = inter.groupby('customer_id').week.nth(-1)
    test_candidates['customer_id_last_time'] = test_candidates['customer_id'].map(tmp)

    tmp = inter.groupby('customer_id').week.nth(0)
    test_candidates['customer_id_first_time'] = test_candidates['customer_id'].map(tmp)

    tmp = inter.groupby('customer_id').week.mean()
    test_candidates['customer_id_time_mean'] = test_candidates['customer_id'].map(tmp)

    test_candidates['customer_id_gap'] = test_candidates['customer_id_first_time'] - test_candidates['customer_id_last_time']

    gc.collect()
    # * ==================================================
    
    batch_size = 5_000_000
    probs = np.zeros(test_candidates.shape[0])
    for batch in tqdm(range(0, test_candidates.shape[0], batch_size)):
        outputs = ranker.predict(test_candidates.loc[batch : batch + batch_size - 1, feats])
        probs[batch : batch + batch_size] = outputs
    test_candidates["prob"] = probs
    pred_lgb = test_candidates[['customer_id','article_id','prob']]
    pred_lgb = pred_lgb.sort_values(by=["customer_id","prob"], ascending=False).reset_index(drop=True)
    pred_lgb.rename(columns={'article_id':'prediction'}, inplace=True)
    pred_lgb = pred_lgb.drop_duplicates(['customer_id', 'prediction'], keep='first')
    pred_lgb['customer_id'] = pred_lgb['customer_id'].astype(int)
    test_pred.append(pred_lgb)
    del test_candidates
    gc.collect()

Chunk 0


53

100%|██████████| 6/6 [02:57<00:00, 29.52s/it]


24

Chunk 1


0

100%|██████████| 6/6 [02:26<00:00, 24.38s/it]


4

In [ ]:
pred_lgb = pd.concat(test_pred, ignore_index=True)

In [ ]:
# pred_lgb.to_parquet(data_dir/"processed"/"small_binary_test.pqt")
pred_lgb.to_parquet(data_dir/"processed"/"small_rank_test.pqt")